In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np



Using TensorFlow backend.


In [2]:
from keras import applications
def vgg16Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = applications.VGG16(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [3]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def restNet_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [5]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

def inceptionV3_model(image_size = (224, 224), if_draw_model = False):

    width = image_size[0]
    height = image_size[1]
    
    base_model = InceptionV3(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = True
        
    for i, layer in enumerate(base_model.layers):
        print(i, layer.name)
        
   # for layer in base_model.layers[25:]:
   #     layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [32]:
#简单模型
def simple_model(time_len=1):
    ch, row, col = 3, 66, 200  # camera format
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
  #  model.add(Lambda(lambda x: x,
            input_shape=( row, col, ch),
            output_shape=( row, col,ch)))
    model.add(Convolution2D(3, 3, 3, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    
    model.add(Flatten())
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(10))
#  model.add(Lambda(nor_output_1))
    sgd = optimizers.SGD(lr=0.00003, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)

      

    return model

In [6]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()

In [4]:
import os
import shutil
import pandas as pd
import cv2
from sklearn.utils import shuffle
np.random.seed(2017)

def get_driver_list():
    driver_id_list = []
    df = pd.read_csv("driver_imgs_list.csv")
    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        is_saved_id = False
        for saved_id in driver_id_list:
            if saved_id == driver_id:
                is_saved_id = True
                break
        
        if is_saved_id == False:
            driver_id_list.append(driver_id)
    return driver_id_list

In [9]:

def prepare_data():
    dirbase = 'imgs/train2/'
    if os.path.exists(dirbase):
        return
    os.mkdir(dirbase)
        
    driver_id_list = get_driver_list()
    
    df = pd.read_csv("driver_imgs_list.csv")  
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for driver_id in driver_id_list:
        os.mkdir(dirbase + driver_id)
        for sub_dir in sub_dirs:
            os.mkdir(dirbase + driver_id + '/' + sub_dir)

    for i in range(df.shape[0]):
        driver_id = df.loc[i]["subject"]
        class_name = df.loc[i]["classname"]
        img = df.loc[i]["img"]
        add_old = 'imgs/train/' + class_name+'/' + img
        add_new = dirbase + driver_id + '/' + class_name + '/' + img
        if os.path.exists(add_old):
            shutil.move(add_old, add_new)
        
prepare_data()    

In [5]:
image_size=(224, 224)
batch_size =64

In [6]:
#训练
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg19 import preprocess_input
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions

import copy
def get_file_num(driver_id):
    train_base_add = 'imgs/train2/'
    add_driver = train_base_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    num = 0
    for sub_dir in sub_dirs:
        add_driver_class = add_driver + '/' + sub_dir
        train_filenames = os.listdir(add_driver_class)
        num += len(train_filenames)
    return num

def train_model(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 5
    loss_list = []
    
    best_model = 0
    save_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:loss_list.append(logs['val_loss']))
                            
    for epoch in range(10):
        print('step---------------------- {%d}'%(epoch))
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            train_generator = train_datagen.flow_from_directory(
                train_base_add + driver_id,
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
    
            val_generator = train_datagen.flow_from_directory(
                'imgs/val',
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical')
        
            model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=1,
                workers=6,
                callbacks=[save_loss_callback],
                validation_data = val_generator)
            
            last_loss = loss_list[-1]
            if_best_loss = True
            for loss in loss_list:
                if loss < last_loss:
                    if_best_loss = False
            if if_best_loss == True:
            #    best_model =copy.deepcopy(model) 
                print('last model loss ------------------------------------------%f'%(last_loss))
                model.save('model.h5')
    
    
    model = load_model('model.h5')
    
    return model


In [7]:
#训练方式2
from keras.callbacks import *
from keras.models import *
#from keras.applications.vgg19 import preprocess_input
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.inception_v3 import preprocess_input, decode_predictions
from sklearn.preprocessing import OneHotEncoder
from skimage.io import imread, imsave
from scipy.misc import imresize

def load_image(path):
    img = imread(path)
    img = imresize(img, (image_size[1], image_size[0]))
    return img

def load_driver_imgs(driver_id, pre_add = 'imgs/train2/'):
    base_add = pre_add + driver_id
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    
    img_list = []
    label_list = []
    for i, sub_dir in enumerate(sub_dirs) :
        img_class_add = base_add + '/' + sub_dir
        img_names = os.listdir(img_class_add)
        for img_name in img_names:
            
            img = load_image(img_class_add + '/' + img_name)
       #     img = img.swapaxes(2, 0)
            img_list.append(img)
            label_val = [0,0,0,0,0,0,0,0,0,0]
            label_val[i] = 1
            label_list.append(label_val)
    
    img_list = np.array(img_list)
    label_list = np.array(label_list)
  #  label_list = OneHotEncoder(n_values=10).fit_transform(label_list.reshape(-1, 1)).toarray()
    return img_list, label_list

def train_model_2(Model):
    model = Model()
    
    train_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    driver_id_list = get_driver_list()
    train_base_add = 'imgs/train2/'
    
    
    patience = 5
    loss_list = []
    
    best_model = model
    save_loss_callback = LambdaCallback(on_epoch_end=lambda epoch, logs:loss_list.append(logs['val_loss']))
    
    val_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    x_val, y_val = load_driver_imgs('val', 'imgs/')
    val_datagen.fit(x_val)
    
    
    for epoch in range(10):
        print('step---------------------- {%d}'%(epoch))
        for driver_id in driver_id_list:
            nb_train_samples = get_file_num(driver_id)
            
            x_train, y_train = load_driver_imgs(driver_id)
        
            train_datagen.fit(x_train)
            model.fit_generator(
                train_datagen.flow(x_train, y_train, batch_size=batch_size),
                shuffle=True,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=1,
                workers=6,
                callbacks=[save_loss_callback],
                validation_data = val_datagen.flow(x_val, y_val, batch_size=batch_size)
               
            )
            
            last_loss = loss_list[-1]
            if_best_loss = True
            for loss in loss_list:
                if loss < last_loss:
                    if_best_loss = False
            if if_best_loss == True:
                best_model = model
                print('last model loss ------------------------------------------%f'%(last_loss))
                model.save('model.h5')
    
    
    model = load_model('model.h5')
    return model


In [8]:
import pandas as pd
import cv2
def test_model(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'
    test_filenames = os.listdir(test_address)
 #   test_filenames = test_filenames[:10]
    X = np.zeros((len(test_filenames), image_size[0], image_size[1], 3), dtype=np.uint8)

    for i, test_file in enumerate(test_filenames) :
        X[i] = cv2.resize(cv2.imread(test_address + '/' + test_file), image_size)
    
    y_pred = model.predict(X, batch_size=batch_size, verbose=0)
  #  print(y_pred)
    
    for i, fname in enumerate(test_filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)
    

In [9]:
import pandas as pd
import cv2
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input, decode_predictions

def test_model_2(model, file_name):
    df = pd.read_csv("sample_submission.csv")
    
    test_address = 'imgs/test'

    
    test_datagen = ImageDataGenerator(
 #   shear_range=0.2,
 #   zoom_range=0.2,
    preprocessing_function=preprocess_input,
 #   horizontal_flip=True
    )
    test_generator = test_datagen.flow_from_directory(
        test_address,
        target_size=(image_size[0], image_size[1]),
        shuffle = "false",
        class_mode='categorical',
        batch_size=1)

    filenames = test_generator.filenames
    nb_samples = len(filenames)
   
    y_pred = model.predict_generator(test_generator, steps = nb_samples)
  #  print(y_pred)
    
    for i, fname in enumerate(filenames):
      #  print(fname)
        df.set_value(i, 'img', fname)
        sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
        for ii, sb in enumerate(sub_dirs):
            df.set_value(i, sb, y_pred[i][ii])


    df.to_csv(file_name, index=None)
    df.head(10)

In [ ]:
model = train_model_2(restNet_model)
test_model(model, 'pred.csv')

0 input_1
1 conv1
2 bn_conv1
3 activation_1
4 max_pooling2d_1
5 res2a_branch2a
6 bn2a_branch2a
7 activation_2
8 res2a_branch2b
9 bn2a_branch2b
10 activation_3
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_1
16 activation_4
17 res2b_branch2a
18 bn2b_branch2a
19 activation_5
20 res2b_branch2b
21 bn2b_branch2b
22 activation_6
23 res2b_branch2c
24 bn2b_branch2c
25 add_2
26 activation_7
27 res2c_branch2a
28 bn2c_branch2a
29 activation_8
30 res2c_branch2b
31 bn2c_branch2b
32 activation_9
33 res2c_branch2c
34 bn2c_branch2c
35 add_3
36 activation_10
37 res3a_branch2a
38 bn3a_branch2a
39 activation_11
40 res3a_branch2b
41 bn3a_branch2b
42 activation_12
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_4
48 activation_13
49 res3b_branch2a
50 bn3b_branch2a
51 activation_14
52 res3b_branch2b
53 bn3b_branch2b
54 activation_15
55 res3b_branch2c
56 bn3b_branch2c
57 add_5
58 activation_16
59 res3c_branch2a
60 bn3c_branch2a
61 activation_17
62 res3c

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


step---------------------- {0}
Epoch 1/1
10/10 [==============================] - 213s 21s/step - loss: 1.6884 - acc: 0.4607 - val_loss: 5.3382 - val_acc: 0.2034
last model loss ------------------------------------------5.338233
Epoch 1/1
11/11 [==============================] - 8s 743ms/step - loss: 2.2889 - acc: 0.3526 - val_loss: 7.7284 - val_acc: 0.1216
Epoch 1/1
12/12 [==============================] - 8s 685ms/step - loss: 2.0086 - acc: 0.5262 - val_loss: 13.1990 - val_acc: 0.1596
Epoch 1/1
12/12 [==============================] - 8s 695ms/step - loss: 1.1578 - acc: 0.6433 - val_loss: 14.4408 - val_acc: 0.1037
Epoch 1/1
14/14 [==============================] - 9s 656ms/step - loss: 0.7741 - acc: 0.6912 - val_loss: 14.4529 - val_acc: 0.1033
Epoch 1/1
17/17 [==============================] - 10s 591ms/step - loss: 0.6168 - acc: 0.7954 - val_loss: 13.2743 - val_acc: 0.1118
Epoch 1/1
17/17 [==============================] - 10s 591ms/step - loss: 0.2941 - acc: 0.8823 - val_loss: 8.56

In [47]:
model = train_model(restNet_model)

0 input_14
1 conv1
2 bn_conv1
3 activation_638
4 max_pooling2d_14
5 res2a_branch2a
6 bn2a_branch2a
7 activation_639
8 res2a_branch2b
9 bn2a_branch2b
10 activation_640
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_209
16 activation_641
17 res2b_branch2a
18 bn2b_branch2a
19 activation_642
20 res2b_branch2b
21 bn2b_branch2b
22 activation_643
23 res2b_branch2c
24 bn2b_branch2c
25 add_210
26 activation_644
27 res2c_branch2a
28 bn2c_branch2a
29 activation_645
30 res2c_branch2b
31 bn2c_branch2b
32 activation_646
33 res2c_branch2c
34 bn2c_branch2c
35 add_211
36 activation_647
37 res3a_branch2a
38 bn3a_branch2a
39 activation_648
40 res3a_branch2b
41 bn3a_branch2b
42 activation_649
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_212
48 activation_650
49 res3b_branch2a
50 bn3b_branch2a
51 activation_651
52 res3b_branch2b
53 bn3b_branch2b
54 activation_652
55 res3b_branch2c
56 bn3b_branch2c
57 add_213
58 activation_653
59 res3c_branch2a
60 bn

4/4 [==============================] - 7s 2s/step - loss: 1.9095 - acc: 0.3867 - val_loss: 9.8907 - val_acc: 0.1833
Found 734 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 847ms/step - loss: 0.7188 - acc: 0.7754 - val_loss: 11.6367 - val_acc: 0.1422
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 839ms/step - loss: 0.5455 - acc: 0.8433 - val_loss: 10.6182 - val_acc: 0.1042
step---------------------- {1}
Found 655 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 913ms/step - loss: 0.7516 - acc: 0.7531 - val_loss: 4.6243 - val_acc: 0.1667
Found 749 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 10s 865ms/step - loss: 0.7958 - acc: 0.7620 - val_loss: 2.9529 - va

Found 754 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 843ms/step - loss: 0.8262 - acc: 0.7745 - val_loss: 2.1472 - val_acc: 0.4765
last model loss ------------------------------------------2.147226
Found 596 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
9/9 [==============================] - 8s 938ms/step - loss: 0.3070 - acc: 0.8972 - val_loss: 8.9202 - val_acc: 0.1149
Found 544 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.1822 - acc: 0.9456 - val_loss: 11.1202 - val_acc: 0.0975
Found 540 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.2344 - acc: 0.9311 - val_loss: 8.2619 - val_acc: 0.1703
Found 656 images belonging to 10 classes.
Found 2237 images belonging to 10 clas

Found 733 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 834ms/step - loss: 0.2579 - acc: 0.9141 - val_loss: 2.7018 - val_acc: 0.5262
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 850ms/step - loss: 0.3276 - acc: 0.8777 - val_loss: 2.7294 - val_acc: 0.5181
Found 922 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 10s 738ms/step - loss: 0.2862 - acc: 0.9067 - val_loss: 2.7137 - val_acc: 0.5369
Found 314 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
4/4 [==============================] - 7s 2s/step - loss: 0.2817 - acc: 0.9228 - val_loss: 2.5684 - val_acc: 0.5601
Found 734 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 8

14/14 [==============================] - 10s 743ms/step - loss: 0.0509 - acc: 0.9866 - val_loss: 3.2440 - val_acc: 0.4524
Found 1121 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
17/17 [==============================] - 11s 657ms/step - loss: 0.1237 - acc: 0.9641 - val_loss: 5.1913 - val_acc: 0.3527
Found 1123 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
17/17 [==============================] - 11s 665ms/step - loss: 0.0535 - acc: 0.9822 - val_loss: 1.3321 - val_acc: 0.7108
Found 1109 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
17/17 [==============================] - 11s 654ms/step - loss: 0.0392 - acc: 0.9917 - val_loss: 0.8600 - val_acc: 0.7774
Found 1071 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
16/16 [==============================] - 11s 686ms/step - loss: 0.0459 - acc: 0.9873 - val_loss: 0.7482 - val_acc: 0.8006
last m

11/11 [==============================] - 9s 857ms/step - loss: 0.0296 - acc: 0.9929 - val_loss: 1.2364 - val_acc: 0.7720
Found 911 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 11s 753ms/step - loss: 0.5957 - acc: 0.8460 - val_loss: 6.6107 - val_acc: 0.3657
Found 708 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 858ms/step - loss: 0.2233 - acc: 0.9418 - val_loss: 7.8577 - val_acc: 0.2494
Found 832 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
13/13 [==============================] - 10s 780ms/step - loss: 0.1761 - acc: 0.9531 - val_loss: 4.2768 - val_acc: 0.3907
Found 651 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 916ms/step - loss: 0.0828 - acc: 0.9766 - val_loss: 2.5646 - val_acc: 0.5449
Found 717 ima

step---------------------- {8}
Found 655 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 883ms/step - loss: 0.0376 - acc: 0.9843 - val_loss: 0.4802 - val_acc: 0.8717
Found 749 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 856ms/step - loss: 0.8037 - acc: 0.7629 - val_loss: 6.8792 - val_acc: 0.1301
Found 778 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
12/12 [==============================] - 9s 789ms/step - loss: 0.1357 - acc: 0.9581 - val_loss: 4.9116 - val_acc: 0.3849
Found 792 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
12/12 [==============================] - 10s 797ms/step - loss: 0.1102 - acc: 0.9686 - val_loss: 2.6081 - val_acc: 0.5561
Found 957 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==

8/8 [==============================] - 8s 1s/step - loss: 0.0451 - acc: 0.9882 - val_loss: 2.1553 - val_acc: 0.6165
Found 540 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.0266 - acc: 0.9941 - val_loss: 1.6265 - val_acc: 0.7059
Found 656 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 884ms/step - loss: 0.1012 - acc: 0.9769 - val_loss: 2.2448 - val_acc: 0.6477
Found 748 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 852ms/step - loss: 0.1048 - acc: 0.9692 - val_loss: 2.3029 - val_acc: 0.6634
Found 911 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 10s 726ms/step - loss: 0.1578 - acc: 0.9619 - val_loss: 5.5484 - val_acc: 0.4131
Found 708 images belongi

In [48]:
test_model(model, 'pred.csv')

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [52]:
test_model_2(model, 'pred.csv')

Found 0 images belonging to 0 classes.


In [29]:
model = train_model(restNet_model)

0 input_6
1 conv1
2 bn_conv1
3 activation_246
4 max_pooling2d_6
5 res2a_branch2a
6 bn2a_branch2a
7 activation_247
8 res2a_branch2b
9 bn2a_branch2b
10 activation_248
11 res2a_branch2c
12 res2a_branch1
13 bn2a_branch2c
14 bn2a_branch1
15 add_81
16 activation_249
17 res2b_branch2a
18 bn2b_branch2a
19 activation_250
20 res2b_branch2b
21 bn2b_branch2b
22 activation_251
23 res2b_branch2c
24 bn2b_branch2c
25 add_82
26 activation_252
27 res2c_branch2a
28 bn2c_branch2a
29 activation_253
30 res2c_branch2b
31 bn2c_branch2b
32 activation_254
33 res2c_branch2c
34 bn2c_branch2c
35 add_83
36 activation_255
37 res3a_branch2a
38 bn3a_branch2a
39 activation_256
40 res3a_branch2b
41 bn3a_branch2b
42 activation_257
43 res3a_branch2c
44 res3a_branch1
45 bn3a_branch2c
46 bn3a_branch1
47 add_84
48 activation_258
49 res3b_branch2a
50 bn3b_branch2a
51 activation_259
52 res3b_branch2b
53 bn3b_branch2b
54 activation_260
55 res3b_branch2c
56 bn3b_branch2c
57 add_85
58 activation_261
59 res3c_branch2a
60 bn3c_bran

Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 832ms/step - loss: 0.7194 - acc: 0.7728 - val_loss: 13.7494 - val_acc: 0.1167
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 809ms/step - loss: 0.7150 - acc: 0.7938 - val_loss: 12.5258 - val_acc: 0.1685
step {1}
Found 655 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 859ms/step - loss: 0.5128 - acc: 0.8477 - val_loss: 13.6141 - val_acc: 0.1381
Found 749 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 825ms/step - loss: 1.0530 - acc: 0.7049 - val_loss: 12.3854 - val_acc: 0.1225
Found 778 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
12/12 [==============================] - 9s 765ms/step - loss: 0.5946 

Found 596 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
9/9 [==============================] - 9s 945ms/step - loss: 0.2362 - acc: 0.9149 - val_loss: 1.5921 - val_acc: 0.6187
Found 544 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.1771 - acc: 0.9451 - val_loss: 1.5005 - val_acc: 0.6388
Found 540 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.1275 - acc: 0.9657 - val_loss: 1.3337 - val_acc: 0.6710
Found 656 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 892ms/step - loss: 0.1453 - acc: 0.9719 - val_loss: 1.5156 - val_acc: 0.6361
Found 748 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 815ms/ste

Found 922 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 10s 718ms/step - loss: 0.2283 - acc: 0.9376 - val_loss: 2.6392 - val_acc: 0.5516
Found 314 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
4/4 [==============================] - 7s 2s/step - loss: 0.4075 - acc: 0.8525 - val_loss: 2.3578 - val_acc: 0.6232
Found 734 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 825ms/step - loss: 0.2140 - acc: 0.9444 - val_loss: 2.9189 - val_acc: 0.5494
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 810ms/step - loss: 0.1779 - acc: 0.9619 - val_loss: 1.8849 - val_acc: 0.5695
step {4}
Found 655 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [=============================

Found 1071 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
16/16 [==============================] - 11s 677ms/step - loss: 0.2562 - acc: 0.9111 - val_loss: 5.0764 - val_acc: 0.4283
Found 754 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 833ms/step - loss: 0.0926 - acc: 0.9815 - val_loss: 2.9109 - val_acc: 0.5507
Found 596 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
9/9 [==============================] - 8s 922ms/step - loss: 0.1245 - acc: 0.9702 - val_loss: 1.3999 - val_acc: 0.7135
Found 544 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/step - loss: 0.1317 - acc: 0.9650 - val_loss: 1.4261 - val_acc: 0.7135
Found 540 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
8/8 [==============================] - 8s 1s/s

Found 733 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 815ms/step - loss: 0.1414 - acc: 0.9613 - val_loss: 6.3784 - val_acc: 0.2937
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 831ms/step - loss: 0.0990 - acc: 0.9688 - val_loss: 2.2471 - val_acc: 0.6267
Found 922 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 10s 717ms/step - loss: 0.0995 - acc: 0.9709 - val_loss: 1.2654 - val_acc: 0.7479
Found 314 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
4/4 [==============================] - 7s 2s/step - loss: 0.1338 - acc: 0.9607 - val_loss: 0.8640 - val_acc: 0.8002
Found 734 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 8

Found 1123 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
17/17 [==============================] - 11s 640ms/step - loss: 0.0218 - acc: 0.9926 - val_loss: 0.6390 - val_acc: 0.8333
Found 1109 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
17/17 [==============================] - 11s 643ms/step - loss: 0.0302 - acc: 0.9917 - val_loss: 0.5779 - val_acc: 0.8578
Found 1071 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
16/16 [==============================] - 11s 677ms/step - loss: 0.0260 - acc: 0.9941 - val_loss: 0.5156 - val_acc: 0.8637
Found 754 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 832ms/step - loss: 0.0297 - acc: 0.9943 - val_loss: 0.4156 - val_acc: 0.8824
Found 596 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
9/9 [==============================

Found 651 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
10/10 [==============================] - 9s 891ms/step - loss: 0.0490 - acc: 0.9875 - val_loss: 1.9806 - val_acc: 0.6679
Found 717 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 795ms/step - loss: 0.0459 - acc: 0.9820 - val_loss: 0.9513 - val_acc: 0.7725
Found 733 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 821ms/step - loss: 0.0334 - acc: 0.9856 - val_loss: 0.8348 - val_acc: 0.8015
Found 736 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
11/11 [==============================] - 9s 800ms/step - loss: 0.0457 - acc: 0.9873 - val_loss: 0.7710 - val_acc: 0.8002
Found 922 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/1
14/14 [==============================] - 

In [30]:
test_model(model, 'pred.csv')

/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/ubuntu/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
